### Data file preparation steps:
    
    * The raw text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file llist.txt is then processed through the bash script file a_updater.sh to add commas at the appropriate locations and make other modifications, by typing ./a_updater into Terminal - ready to be imported from within Jupyter.

# Preview R15 of 2019 season

In [1]:
#%%html
#<img src="img/ess_stk.png" alt="Drawing" align="left" style="width: 800px;"/>

In [2]:
import os 
os.getcwd() 

'/Users/stevegabriel/data_projects/sports_data/afl_dataexploration'

In [3]:
import pandas as pd

In [4]:
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

#### The file will contain all matches starting from the first ever VFL game in 1987 - more than fifteen thousand games.

In [5]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [6]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15520,15520,21-Jun-2019,R14,Sydney,12,10,82,Hawthorn,9,9,63,S.C.G.
15521,15521,22-Jun-2019,R14,Melbourne,13,7,85,Fremantle,11,5,71,M.C.G.
15522,15522,22-Jun-2019,R14,St Kilda,8,11,59,Brisbane Lions,17,13,115,Docklands
15523,15523,22-Jun-2019,R14,Port Adelaide,9,13,67,Geelong,8,8,56,Adelaide Oval
15524,15524,23-Jun-2019,R14,Western Bulldogs,10,13,73,Collingwood,13,4,82,Docklands


#### Shown above, the most four finals matches played on the weekend.

### Adding additional information including state, and addition team names as two letter codes

In [7]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [8]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [9]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13098
WA        655
SA        586
NSW       523
QLD       486
TAS        95
ACT        50
NT         25
INTL        6
Name: state, dtype: int64

In [10]:
## 1. HVAR & AGGR
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2']

In [11]:
## 2. YEAR & MONTH
import time
import datetime

In [13]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

In [14]:
## 3. ABBREVIATED CLUB NAMES
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [15]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [16]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [17]:
trans['t1'].value_counts().sum()

15524

##  FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [18]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [19]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [20]:
trans["team2"].value_counts()

Collingwood         1281
Carlton             1258
Essendon            1229
Geelong             1226
Melbourne           1197
St Kilda            1196
Richmond            1126
North Melbourne      992
Hawthorn             987
Western Bulldogs     981
Fitzroy              960
South Melbourne      788
Sydney               438
West Coast           382
Brisbane             366
Adelaide             331
Fremantle            280
Port Adelaide        261
Gold Coast            94
GWS Giants            88
University            63
Name: team2, dtype: int64

#### The dataframe with the additional column. Currently it has one game per row.

In [21]:
trans.head(10)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state,hvar,aggr,year,month,t1,t2
1,1,1897-05-08,R1,Fitzroy,6,13,49,Carlton,2,4,16,Brunswick St,VIC,33,65,1897,5,FI,CA
2,2,1897-05-08,R1,Collingwood,5,11,41,St Kilda,2,4,16,Victoria Park,VIC,25,57,1897,5,CO,SK
3,3,1897-05-08,R1,Geelong,3,6,24,Essendon,7,5,47,Corio Oval,VIC,-23,71,1897,5,GE,ES
4,4,1897-05-08,R1,South Melbourne,3,9,27,Melbourne,6,8,44,Lake Oval,VIC,-17,71,1897,5,SM,ME
5,5,1897-05-15,R2,South Melbourne,6,4,40,Carlton,5,6,36,Lake Oval,VIC,4,76,1897,5,SM,CA
6,6,1897-05-15,R2,Essendon,4,6,30,Collingwood,8,2,50,East Melbourne,VIC,-20,80,1897,5,ES,CO
7,7,1897-05-15,R2,St Kilda,3,8,26,Fitzroy,10,6,66,Junction Oval,VIC,-40,92,1897,5,SK,FI
8,8,1897-05-15,R2,Melbourne,9,10,64,Geelong,3,1,19,M.C.G.,VIC,45,83,1897,5,ME,GE
9,9,1897-05-22,R3,Collingwood,6,5,41,Geelong,5,7,37,Victoria Park,VIC,4,78,1897,5,CO,GE
10,10,1897-05-22,R3,Fitzroy,5,9,39,Melbourne,7,8,50,Brunswick St,VIC,-11,89,1897,5,FI,ME


## ALL DRAWS (BOOLEAN)

In [22]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [23]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [24]:
trans['draw_rt'].value_counts()

0    15366
1      158
Name: draw_rt, dtype: int64

### The 158 draws above, represent the VFL/AFL games that have concluded in a draw result

### The two ET (extra-time) draws below are statistically relevent, since they also ended regular full-time with both teams on the same score. However extra-time was applied to create a result, due to finals constraints.

In [25]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [26]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15522
1        2
Name: etime_draw, dtype: int64

In [27]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15364
1      160
Name: draws_all, dtype: int64

In [28]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


In [29]:
# 6. GAME CATEGORY
finals = ['EF','QF','SF','PF','GF']

In [30]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [31]:
trans["type"] = trans["round"].apply(game_cat)

In [32]:
trans['type'].value_counts()

reg_season    14869
final           655
Name: type, dtype: int64

### So, there have been 650 finals up to the end of week one of 2018 finals series.

In [33]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [34]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [35]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [36]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

In [37]:
# Export the data frame for use in afl draws EDA
trans.to_csv('all_afl_wide.csv')

## ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [38]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [39]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [40]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [41]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
15516,15516,2019-06-15,R13,Fremantle,15,10,100,Port Adelaide,12,7,...,FR,PA,0,0,0,reg_season,8,16,1,2
15517,15517,2019-06-15,R13,Carlton,15,10,100,Western Bulldogs,15,13,...,CA,WB,0,0,0,reg_season,8,16,2,1
15518,15518,2019-06-16,R13,North Melbourne,10,8,68,GWS Giants,14,7,...,NM,GW,0,0,0,reg_season,8,16,2,1
15519,15519,2019-06-20,R14,West Coast,14,22,106,Essendon,11,5,...,WC,ES,0,0,0,reg_season,8,16,1,2
15520,15520,2019-06-21,R14,Sydney,12,10,82,Hawthorn,9,9,...,SY,HA,0,0,0,reg_season,8,16,1,2
15521,15521,2019-06-22,R14,Melbourne,13,7,85,Fremantle,11,5,...,ME,FR,0,0,0,reg_season,8,16,1,2
15522,15522,2019-06-22,R14,St Kilda,8,11,59,Brisbane,17,13,...,SK,BR,0,0,0,reg_season,8,16,2,1
15523,15523,2019-06-22,R14,Port Adelaide,9,13,67,Geelong,8,8,...,PA,GE,0,0,0,reg_season,8,16,1,2
15524,15524,2019-06-23,R14,Western Bulldogs,10,13,73,Collingwood,13,4,...,WB,CO,0,0,0,reg_season,8,16,2,1


## STAGE TWO
## 9. Reduce and re-order the data frame to prepare for for Long format

In [42]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15520,2019,Sydney,82,Hawthorn,63,S.C.G.,SY,HA,19
15521,2019,Melbourne,85,Fremantle,71,M.C.G.,ME,FR,14
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9


### Create two new columns with values 1 and 0 for the two subsets

In [43]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
tracker.tail(9)

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
15516,2019,Fremantle,100,Port Adelaide,79,Perth Stadium,FR,PA,21,1,0
15517,2019,Carlton,100,Western Bulldogs,103,Docklands,CA,WB,-3,1,0
15518,2019,North Melbourne,68,GWS Giants,91,Bellerive Oval,NM,GW,-23,1,0
15519,2019,West Coast,106,Essendon,71,Perth Stadium,WC,ES,35,1,0
15520,2019,Sydney,82,Hawthorn,63,S.C.G.,SY,HA,19,1,0
15521,2019,Melbourne,85,Fremantle,71,M.C.G.,ME,FR,14,1,0
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1,0
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1,0
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1,0


In [45]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [46]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [47]:
left_team.shape

(15524, 10)

In [48]:
right_team.shape

(15524, 10)

# Rename columns in each dframe 

In [49]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [50]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15520,2019,Hawthorn,63,Sydney,82,S.C.G.,HA,SY,19,0
15521,2019,Fremantle,71,Melbourne,85,M.C.G.,FR,ME,14,0
15522,2019,Brisbane,115,St Kilda,59,Docklands,BR,SK,-56,0
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0


In [51]:
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15520,2019,Hawthorn,63,Sydney,82,S.C.G.,HA,SY,19,0
15521,2019,Fremantle,71,Melbourne,85,M.C.G.,FR,ME,14,0
15522,2019,Brisbane,115,St Kilda,59,Docklands,BR,SK,-56,0
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0


In [52]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15520,2019,Sydney,82,Hawthorn,63,S.C.G.,SY,HA,19,1
15521,2019,Melbourne,85,Fremantle,71,M.C.G.,ME,FR,14,1
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1


## CONCATENATION

### Can now concatenate the two dframes, as in si_week11

In [53]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


#### The new long data frame consisting of 30,000 plus rows:

In [54]:
merge_ft.shape

(31048, 10)

# The following is to keep each game together (two rows per game)

In [55]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


In [56]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0


## There are actually 30,804 rows, indexing isn't  sequential

## Create team result column

In [57]:
def neg_hvar(hvar):
        return -1*hvar

In [58]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [59]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1,56
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0,-11
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1,-11
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1,9
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0,9


In [60]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [61]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [62]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [63]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1,56,1,-56
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0,-11,-1,-11
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1,-11,1,11
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1,9,1,-9
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0,9,-1,9


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [64]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [65]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


### Note that the column 'lr' denotes left right, so there are only two different values (1 denotes left)

In [66]:
merge_ft['lr'].value_counts()

1    15524
0    15524
Name: lr, dtype: int64

## New function for win/loss/draw (W/L/D)


In [67]:
def outcome(tm_var):
    if tm_var >= 1:
        return "W"
    elif tm_var <= -1:
        return "L"
    elif tm_var == 0:
        return "D"
    else: return "Unknown"

In [68]:
merge_ft["outcome"] = merge_ft['tm_var'].apply(outcome)

In [69]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33,W
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33,L
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25,W
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25,L
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23,L


In [70]:
merge_ft.tail(9)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15520,2019,Sydney,82,Hawthorn,63,S.C.G.,SY,HA,19,1,19,W
15521,2019,Fremantle,71,Melbourne,85,M.C.G.,FR,ME,14,0,-14,L
15521,2019,Melbourne,85,Fremantle,71,M.C.G.,ME,FR,14,1,14,W
15522,2019,Brisbane,115,St Kilda,59,Docklands,BR,SK,-56,0,56,W
15522,2019,St Kilda,59,Brisbane,115,Docklands,SK,BR,-56,1,-56,L
15523,2019,Geelong,56,Port Adelaide,67,Adelaide Oval,GE,PA,11,0,-11,L
15523,2019,Port Adelaide,67,Geelong,56,Adelaide Oval,PA,GE,11,1,11,W
15524,2019,Western Bulldogs,73,Collingwood,82,Docklands,WB,CO,-9,1,-9,L
15524,2019,Collingwood,82,Western Bulldogs,73,Docklands,CO,WB,-9,0,9,W


## Research for Week 15 of 2019 using Pandas filtering:
### * Essendon v GWS
### * Hawthorn v WC

### Recent contests ESS v GWS

In [72]:
merge_ft.iloc[:,:][(merge_ft.tm == 'ES') & (merge_ft.op == 'GW')].tail(8)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
14213,2013,Essendon,120,GWS Giants,81,Docklands,ES,GW,39,1,39,W
14304,2013,Essendon,116,GWS Giants,77,Sydney Showground,ES,GW,-39,0,39,W
14467,2014,Essendon,81,GWS Giants,66,Sydney Showground,ES,GW,-15,0,15,W
14740,2015,Essendon,61,GWS Giants,93,Sydney Showground,ES,GW,32,0,-32,L
14900,2016,Essendon,90,GWS Giants,117,Docklands,ES,GW,-27,1,-27,L
15086,2017,Essendon,101,GWS Giants,117,Sydney Showground,ES,GW,16,0,-16,L
15286,2018,Essendon,95,GWS Giants,60,Sydney Showground,ES,GW,-35,0,35,W
15415,2019,Essendon,40,GWS Giants,112,Sydney Showground,ES,GW,72,0,-72,L


### Recent contests HA v West Coast in Melb 

In [74]:
merge_ft.iloc[:,:][(merge_ft.tm == 'HA') & (merge_ft.op == 'WC') & (merge_ft.venue != 'Subiaco')].tail(12)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
12944,2006,Hawthorn,104,West Coast,114,M.C.G.,HA,WC,-10,1,-10,L
13089,2007,Hawthorn,99,West Coast,64,York Park,HA,WC,35,1,35,W
13312,2008,Hawthorn,126,West Coast,69,M.C.G.,HA,WC,57,1,57,W
13427,2009,Hawthorn,74,West Coast,56,York Park,HA,WC,18,1,18,W
13790,2011,Hawthorn,72,West Coast,65,York Park,HA,WC,7,1,7,W
14149,2012,Hawthorn,95,West Coast,70,M.C.G.,HA,WC,25,1,25,W
14269,2013,Hawthorn,123,West Coast,103,Docklands,HA,WC,20,1,20,W
14465,2014,Hawthorn,123,West Coast,79,York Park,HA,WC,44,1,44,W
14786,2015,Hawthorn,107,West Coast,61,M.C.G.,HA,WC,46,1,46,W
14803,2016,Hawthorn,99,West Coast,53,M.C.G.,HA,WC,46,1,46,W
